# Deep Q-Learning (DQN) Project

This notebook shows how to train an AI agent using Deep Q-Learning to play game like Flappy Bird.

## What is Deep Q-Learning?

Deep Q-Learning uses a neural network to help an AI agent learn how to make good decisions. The agent learns by trying different actions and seeing what rewards it gets.

## Project Structure

1. Experience Replay - Stores past experiences to learn from
2. DQN Network - The brain of the agent
3. Agent - Controls training and decision making
4. Hyperparameters - Settings that control how the agent learns
5. Training - Teaching the agent to play
6. Evaluation - Testing how well the agent learned

## Step 1: Install Required Libraries

First we need to install the libraries we will use. Run this cell to install everything needed.

In [ ]:
# Install the required packages
# You only need to run this once
!pip install gymnasium
!pip install torch
!pip install matplotlib
!pip install pyyaml
!pip install flappy-bird-gymnasium

## Step 2: Import Libraries

Now we import all the libraries we need for the project. These include tools for making the game environment, doing math, creating graphs, and building neural networks.

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import torch
from torch import nn
import torch.nn.functional as F
from collections import deque
from datetime import datetime, timedelta
import itertools
import os

# Try to import Flappy Bird (optional)
try:
    import flappy_bird_gymnasium
    print("Flappy Bird is available!")
except:
    print("Flappy Bird not installed. Only CartPole will work.")

# Use CPU for computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# For saving plots as images
matplotlib.use('Agg')

# Create directory for saving results
RUNS_DIR = "runs"
os.makedirs(RUNS_DIR, exist_ok=True)

## Step 3: Experience Replay Memory

Experience Replay is like a memory bank where the agent stores past experiences. Each experience contains:
- Current state (what the agent saw)
- Action taken (what the agent did)
- New state (what happened after the action)
- Reward (points earned or lost)
- Done flag (whether the game ended)

The agent learns better when it trains on random samples from this memory instead of just learning from the most recent experience. This prevents the agent from forgetting old lessons.

In [ ]:
class ReplayMemory():
    """Stores experiences for the agent to learn from"""
    
    def __init__(self, maxlen, seed=None):
        # Create a memory that can hold a maximum number of experiences
        self.memory = deque([], maxlen=maxlen)
        
        # Optional seed for getting the same random results every time
        if seed is not None:
            random.seed(seed)
    
    def append(self, transition):
        """Add a new experience to memory"""
        self.memory.append(transition)
    
    def sample(self, sample_size):
        """Get random experiences from memory for training"""
        return random.sample(self.memory, sample_size)
    
    def __len__(self):
        """Return how many experiences are stored"""
        return len(self.memory)

## Step 4: Deep Q-Network (DQN)

This is the brain of our agent. It is a neural network that takes the current state as input and outputs Q-values for each possible action.

### What are Q-values?
Q-values tell us how good each action is. The agent picks the action with the highest Q-value.

### Network Architecture Options:

1. **Standard DQN**: Simple network with layers that connect all neurons together
2. **Dueling DQN**: Splits the network into two streams:
   - Value stream: Estimates how good the current state is overall
   - Advantage stream: Estimates how much better each action is compared to average

The Dueling architecture helps the agent learn faster because it can evaluate states and actions separately.

In [ ]:
class DQN(nn.Module):
    """Deep Q-Network - The brain of the agent"""
    
    def __init__(self, state_dim, action_dim, hidden_dim=256, enable_dueling_dqn=True):
        super(DQN, self).__init__()
        
        self.enable_dueling_dqn = enable_dueling_dqn
        
        # First layer: takes the state and processes it
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        
        if self.enable_dueling_dqn:
            # Dueling architecture: two separate streams
            
            # Value stream: how good is this state?
            self.fc_value = nn.Linear(hidden_dim, 256)
            self.value = nn.Linear(256, 1)
            
            # Advantage stream: how good is each action?
            self.fc_advantages = nn.Linear(hidden_dim, 256)
            self.advantages = nn.Linear(256, action_dim)
        else:
            # Standard architecture: direct output
            self.output = nn.Linear(hidden_dim, action_dim)
    
    def forward(self, x):
        """Process the input state and return Q-values"""
        # Pass input through first layer with ReLU activation
        x = F.relu(self.fc1(x))
        
        if self.enable_dueling_dqn:
            # Calculate Value (how good is this state?)
            v = F.relu(self.fc_value(x))
            V = self.value(v)
            
            # Calculate Advantages (how much better is each action?)
            a = F.relu(self.fc_advantages(x))
            A = self.advantages(a)
            
            # Combine Value and Advantages to get Q-values
            # Q = Value + (Advantage - average Advantage)
            Q = V + A - torch.mean(A, dim=1, keepdim=True)
        else:
            # Direct Q-values
            Q = self.output(x)
        
        return Q

## Step 5: DQN Agent

The Agent is the main controller that brings everything together. It handles:
- Loading hyperparameters (settings)
- Training the neural network
- Making decisions (which action to take)
- Saving results and models

In [ ]:
class Agent():
    """Deep Q-Learning Agent that learns to play games"""
    
    def __init__(self, hyperparameters_dict):
        """Initialize the agent with settings"""
        
        # Store all hyperparameters
        self.env_id = hyperparameters_dict['env_id']
        self.learning_rate_a = hyperparameters_dict['learning_rate_a']
        self.discount_factor_g = hyperparameters_dict['discount_factor_g']
        self.network_sync_rate = hyperparameters_dict['network_sync_rate']
        self.replay_memory_size = hyperparameters_dict['replay_memory_size']
        self.mini_batch_size = hyperparameters_dict['mini_batch_size']
        self.epsilon_init = hyperparameters_dict['epsilon_init']
        self.epsilon_decay = hyperparameters_dict['epsilon_decay']
        self.epsilon_min = hyperparameters_dict['epsilon_min']
        self.stop_on_reward = hyperparameters_dict['stop_on_reward']
        self.fc1_nodes = hyperparameters_dict['fc1_nodes']
        self.enable_double_dqn = hyperparameters_dict.get('enable_double_dqn', False)
        self.enable_dueling_dqn = hyperparameters_dict.get('enable_dueling_dqn', True)
        self.env_make_params = hyperparameters_dict.get('env_make_params', {})
        
        # Neural network settings
        self.loss_fn = nn.MSELoss()  # Measures how wrong our predictions are
        self.optimizer = None  # Will be set up during training
        
        # File paths for saving results
        self.experiment_name = hyperparameters_dict.get('name', 'experiment')
        self.LOG_FILE = os.path.join(RUNS_DIR, f'{self.experiment_name}.log')
        self.MODEL_FILE = os.path.join(RUNS_DIR, f'{self.experiment_name}.pt')
        self.GRAPH_FILE = os.path.join(RUNS_DIR, f'{self.experiment_name}.png')
    
    def save_graph(self, rewards_per_episode, epsilon_history):
        """Create and save graphs showing training progress"""
        fig = plt.figure(figsize=(12, 5))
        
        # Graph 1: Show average rewards over time
        mean_rewards = np.zeros(len(rewards_per_episode))
        for x in range(len(mean_rewards)):
            # Calculate average of last 100 episodes
            mean_rewards[x] = np.mean(rewards_per_episode[max(0, x-99):(x+1)])
        
        plt.subplot(121)
        plt.ylabel('Mean Rewards (last 100 episodes)')
        plt.xlabel('Episodes')
        plt.plot(mean_rewards)
        plt.title('Agent Performance Over Time')
        
        # Graph 2: Show how exploration decreases over time
        plt.subplot(122)
        plt.ylabel('Epsilon (Exploration Rate)')
        plt.xlabel('Training Steps')
        plt.plot(epsilon_history)
        plt.title('Exploration vs Exploitation')
        
        plt.tight_layout()
        fig.savefig(self.GRAPH_FILE)
        plt.close(fig)
    
    def optimize(self, mini_batch, policy_dqn, target_dqn):
        """Train the neural network using experiences from memory"""
        
        # Separate the experiences into individual components
        states, actions, new_states, rewards, terminations = zip(*mini_batch)
        
        # Convert to tensors for neural network processing
        states = torch.stack(states)
        actions = torch.stack(actions)
        new_states = torch.stack(new_states)
        rewards = torch.stack(rewards)
        terminations = torch.tensor(terminations).float().to(device)
        
        with torch.no_grad():
            if self.enable_double_dqn:
                # Double DQN: use policy network to choose action, target network to evaluate
                best_actions_from_policy = policy_dqn(new_states).argmax(dim=1)
                target_q = rewards + (1-terminations) * self.discount_factor_g * \
                    target_dqn(new_states).gather(dim=1, index=best_actions_from_policy.unsqueeze(dim=1)).squeeze()
            else:
                # Standard DQN: use target network for both choosing and evaluating
                target_q = rewards + (1-terminations) * self.discount_factor_g * target_dqn(new_states).max(dim=1)[0]
        
        # Get current Q-values from policy network
        current_q = policy_dqn(states).gather(dim=1, index=actions.unsqueeze(dim=1)).squeeze()
        
        # Calculate loss (how wrong are our predictions?)
        loss = self.loss_fn(current_q, target_q)
        
        # Update the network to make better predictions
        self.optimizer.zero_grad()  # Clear old gradients
        loss.backward()  # Calculate new gradients
        self.optimizer.step()  # Update network weights
    
    def run(self, is_training=True, render=False, max_episodes=None):
        """Main function to train or test the agent"""
        
        if is_training:
            start_time = datetime.now()
            last_graph_update_time = start_time
            
            log_message = f"{start_time.strftime('%m-%d %H:%M:%S')}: Training starting..."
            print(log_message)
            with open(self.LOG_FILE, 'w') as file:
                file.write(log_message + '\n')
        
        # Create the game environment
        env = gym.make(self.env_id, render_mode='human' if render else None, **self.env_make_params)
        
        # Get information about the environment
        num_actions = env.action_space.n  # How many actions can the agent take?
        num_states = env.observation_space.shape[0]  # How many numbers describe the state?
        
        # Track rewards for each episode
        rewards_per_episode = []
        
        # Create the policy network (the brain)
        policy_dqn = DQN(num_states, num_actions, self.fc1_nodes, self.enable_dueling_dqn).to(device)
        
        if is_training:
            # Training mode: set up everything needed for learning
            epsilon = self.epsilon_init  # Start with high exploration
            memory = ReplayMemory(self.replay_memory_size)  # Create memory bank
            
            # Create target network (copy of policy network)
            target_dqn = DQN(num_states, num_actions, self.fc1_nodes, self.enable_dueling_dqn).to(device)
            target_dqn.load_state_dict(policy_dqn.state_dict())
            
            # Set up optimizer for training the network
            self.optimizer = torch.optim.Adam(policy_dqn.parameters(), lr=self.learning_rate_a)
            
            epsilon_history = []  # Track exploration rate over time
            step_count = 0  # Count steps for syncing networks
            best_reward = -9999999  # Track best performance
        else:
            # Testing mode: load trained model
            policy_dqn.load_state_dict(torch.load(self.MODEL_FILE))
            policy_dqn.eval()  # Set to evaluation mode
        
        # Determine how many episodes to run
        episodes_to_run = range(max_episodes) if max_episodes else itertools.count()
        
        # Main training/testing loop
        for episode in episodes_to_run:
            # Start a new episode
            state, _ = env.reset()
            state = torch.tensor(state, dtype=torch.float, device=device)
            
            terminated = False
            episode_reward = 0.0
            
            # Play one episode
            while not terminated and episode_reward < self.stop_on_reward:
                # Decide what action to take
                if is_training and random.random() < epsilon:
                    # Explore: take a random action
                    action = env.action_space.sample()
                    action = torch.tensor(action, dtype=torch.int64, device=device)
                else:
                    # Exploit: take the best known action
                    with torch.no_grad():
                        action = policy_dqn(state.unsqueeze(dim=0)).squeeze().argmax()
                
                # Take the action and see what happens
                new_state, reward, terminated, truncated, info = env.step(action.item())
                
                # Add up the rewards
                episode_reward += reward
                
                # Convert to tensors
                new_state = torch.tensor(new_state, dtype=torch.float, device=device)
                reward = torch.tensor(reward, dtype=torch.float, device=device)
                
                if is_training:
                    # Store this experience in memory
                    memory.append((state, action, new_state, reward, terminated))
                    step_count += 1
                
                # Move to next state
                state = new_state
            
            # Episode finished, record the reward
            rewards_per_episode.append(episode_reward)
            
            if is_training:
                # Save model if we got a new best score
                if episode_reward > best_reward:
                    log_message = f"{datetime.now().strftime('%m-%d %H:%M:%S')}: New best reward {episode_reward:.1f} at episode {episode}"
                    print(log_message)
                    with open(self.LOG_FILE, 'a') as file:
                        file.write(log_message + '\n')
                    
                    torch.save(policy_dqn.state_dict(), self.MODEL_FILE)
                    best_reward = episode_reward
                
                # Update graphs periodically
                current_time = datetime.now()
                if current_time - last_graph_update_time > timedelta(seconds=10):
                    self.save_graph(rewards_per_episode, epsilon_history)
                    last_graph_update_time = current_time
                
                # Train the network if we have enough experiences
                if len(memory) > self.mini_batch_size:
                    mini_batch = memory.sample(self.mini_batch_size)
                    self.optimize(mini_batch, policy_dqn, target_dqn)
                    
                    # Reduce exploration over time
                    epsilon = max(epsilon * self.epsilon_decay, self.epsilon_min)
                    epsilon_history.append(epsilon)
                    
                    # Sync networks periodically
                    if step_count > self.network_sync_rate:
                        target_dqn.load_state_dict(policy_dqn.state_dict())
                        step_count = 0
        
        env.close()
        
        if is_training:
            # Final save of graphs
            self.save_graph(rewards_per_episode, epsilon_history)
            print(f"Training completed! Best reward: {best_reward:.1f}")
        
        return rewards_per_episode

## Step 6: Understanding Hyperparameters

Hyperparameters are settings that control how the agent learns. Finding good values for these is called hyperparameter tuning.

### Important Hyperparameters:

1. **Learning Rate** (learning_rate_a): How fast the network learns. Too high and it learns wrong things, too low and it learns very slowly.

2. **Discount Factor** (discount_factor_g): How much the agent cares about future rewards vs immediate rewards. Value between 0 and 1.

3. **Epsilon Decay** (epsilon_decay): Controls how quickly the agent stops exploring and starts using what it learned.

4. **Network Architecture** (fc1_nodes): Number of neurons in the hidden layer. More neurons can learn more complex patterns but train slower.

5. **Batch Size** (mini_batch_size): How many experiences to learn from at once.

6. **Memory Size** (replay_memory_size): How many past experiences to remember.

### Why Different Settings for Different Games?

CartPole is simple so it needs smaller networks and can learn faster. Flappy Bird is harder so it needs bigger networks and more careful tuning.

In [ ]:
# Hyperparameters for CartPole game
cartpole_hyperparameters = {
    'name': 'cartpole_experiment',
    'env_id': 'CartPole-v1',
    'replay_memory_size': 100000,      # Store last 100,000 experiences
    'mini_batch_size': 64,              # Learn from 64 experiences at a time
    'epsilon_init': 1.0,                # Start with 100% random actions
    'epsilon_decay': 0.9995,            # Slowly reduce randomness
    'epsilon_min': 0.01,                # Always keep at least 1% randomness
    'network_sync_rate': 100,           # Sync networks every 100 steps
    'learning_rate_a': 0.001,           # Moderate learning speed
    'discount_factor_g': 0.99,          # Care a lot about future rewards
    'stop_on_reward': 100000,           # Maximum reward before stopping episode
    'fc1_nodes': 128,                   # 128 neurons in hidden layer
    'enable_double_dqn': False,         # Use standard DQN
    'enable_dueling_dqn': True,         # Use dueling architecture
    'env_make_params': {}               # No special environment settings
}

# Hyperparameters for Flappy Bird game
flappybird_hyperparameters = {
    'name': 'flappybird_experiment',
    'env_id': 'FlappyBird-v0',
    'replay_memory_size': 100000,      # Store last 100,000 experiences
    'mini_batch_size': 32,              # Smaller batch size for harder game
    'epsilon_init': 1.0,                # Start with 100% random actions
    'epsilon_decay': 0.99995,           # Decay even slower for harder game
    'epsilon_min': 0.05,                # Keep more exploration for harder game
    'network_sync_rate': 10,            # Sync networks more frequently
    'learning_rate_a': 0.0001,          # Slower learning for stability
    'discount_factor_g': 0.99,          # Care a lot about future rewards
    'stop_on_reward': 100000,           # Maximum reward before stopping episode
    'fc1_nodes': 512,                   # Larger network for harder game
    'enable_double_dqn': True,          # Use Double DQN for better learning
    'enable_dueling_dqn': True,         # Use dueling architecture
    'env_make_params': {
        'use_lidar': False              # Use position values instead of vision
    }
}

## Step 7: Hyperparameter Tuning Experiments

Here we show how to test different hyperparameter combinations to find what works best.

### Grid Search Approach

Grid search means trying different combinations of values for important hyperparameters. We test a few key parameters that have the biggest impact on learning.

In [ ]:
# Define different values to test for important hyperparameters
learning_rates = [0.001, 0.0005, 0.0001]  # Three different learning speeds
hidden_sizes = [64, 128, 256]              # Three different network sizes
discount_factors = [0.95, 0.99]            # Two different future reward weights

# Store results from different experiments
experiment_results = []

print("=" * 60)
print("HYPERPARAMETER TUNING EXPERIMENTS")
print("=" * 60)
print("\nWe will test different combinations of:")
print(f"  - Learning rates: {learning_rates}")
print(f"  - Hidden layer sizes: {hidden_sizes}")
print(f"  - Discount factors: {discount_factors}")
print(f"\nTotal combinations: {len(learning_rates) * len(hidden_sizes) * len(discount_factors)}")
print("\nNote: Each experiment will run for a limited number of episodes.")
print("=" * 60)

## Step 8: Understanding Reward Functions

### What is a Reward Function?

The reward function tells the agent what is good and what is bad. It is the most important part of reinforcement learning because it defines what success means.

### Default Reward Functions in Our Environments:

#### CartPole Rewards:
- **+1 point** for every step the pole stays balanced
- **0 points** when the pole falls or cart goes off screen
- Goal: Keep the pole balanced as long as possible

#### Flappy Bird Rewards:
- **+1 point** for passing through each pipe
- **-1 point** when the bird hits a pipe or the ground
- **+0.1 points** for staying alive (small reward each frame)
- Goal: Fly through as many pipes as possible

### Why Reward Functions Matter:

If rewards are wrong, the agent learns wrong behavior:
- Too much reward for staying alive: agent learns to do nothing
- Too much penalty for mistakes: agent becomes too careful and never tries anything
- Rewards too sparse: agent has trouble learning what actions are good

### Custom Reward Function Example:

Below we show how you could modify rewards if needed. This is useful when default rewards do not give good results.

In [ ]:
def custom_reward_function(env_name, state, action, reward, done, info):
    """
    Example of how to create a custom reward function.
    
    Parameters:
        env_name: Name of the environment
        state: Current state observation
        action: Action that was taken
        reward: Original reward from environment
        done: Whether episode ended
        info: Extra information from environment
    
    Returns:
        Modified reward value
    """
    
    if env_name == 'CartPole-v1':
        # For CartPole, we could add penalty for pole angle
        # state[2] is the pole angle
        angle = abs(state[2])
        
        # Give extra reward for keeping pole more vertical
        if angle < 0.1:
            reward += 0.5  # Bonus for very straight pole
        
        # Penalty for large angles
        if angle > 0.3:
            reward -= 0.5  # Penalty for tilted pole
    
    elif env_name == 'FlappyBird-v0':
        # For Flappy Bird, we could reward staying in middle height
        # Or give bigger reward for passing pipes
        
        if reward > 0.5:  # Likely passed a pipe
            reward *= 2  # Double the reward for passing pipes
        
        if done and reward < 0:  # Hit something
            reward = -10  # Bigger penalty for crashing
    
    return reward

print("Custom reward function defined.")
print("\nThis function can modify rewards to help the agent learn better.")
print("You can adjust it based on what behavior you want to encourage.")

## Step 9: Train the Agent (CartPole Example)

Now we will train an agent to play CartPole. This is a simpler game that trains quickly, so it is good for testing.

### What happens during training:
1. Agent starts by taking random actions (exploration)
2. Agent stores experiences in memory
3. Agent learns from past experiences
4. Agent slowly starts using learned strategy (exploitation)
5. Performance improves over time

Note: Training will run for a limited number of episodes in this notebook. For best results, train for longer (500+ episodes for CartPole).

In [ ]:
# Create and train CartPole agent
print("Creating CartPole agent...")
cartpole_agent = Agent(cartpole_hyperparameters)

print("\nStarting training...")
print("The agent will learn by trial and error.")
print("Watch the rewards increase over time!\n")

# Train for 200 episodes (adjust this number based on your needs)
cartpole_rewards = cartpole_agent.run(is_training=True, render=False, max_episodes=200)

print("\nTraining completed!")
print(f"Final average reward (last 100 episodes): {np.mean(cartpole_rewards[-100:]):.2f}")

## Step 10: Visualize Training Results

Let us look at how the agent improved during training. We will create graphs showing:
1. Rewards over time (did the agent get better?)
2. How long episodes lasted

In [ ]:
# Display the saved training graph
from IPython.display import Image, display
import os

if os.path.exists(cartpole_agent.GRAPH_FILE):
    print("Training Progress Graphs:")
    display(Image(filename=cartpole_agent.GRAPH_FILE))
else:
    print("Training graph not found. Make sure training completed.")

# Also plot raw rewards
plt.figure(figsize=(12, 4))

plt.subplot(131)
plt.plot(cartpole_rewards)
plt.title('Rewards per Episode')
plt.xlabel('Episode')
plt.ylabel('Total Reward')

plt.subplot(132)
# Moving average
window = 50
moving_avg = np.convolve(cartpole_rewards, np.ones(window)/window, mode='valid')
plt.plot(moving_avg)
plt.title(f'Moving Average (window={window})')
plt.xlabel('Episode')
plt.ylabel('Average Reward')

plt.subplot(133)
# Distribution of rewards
plt.hist(cartpole_rewards, bins=30)
plt.title('Distribution of Rewards')
plt.xlabel('Reward')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"\nTraining Statistics:")
print(f"  Total episodes: {len(cartpole_rewards)}")
print(f"  Best reward: {max(cartpole_rewards):.2f}")
print(f"  Average reward: {np.mean(cartpole_rewards):.2f}")
print(f"  Final 100 episodes average: {np.mean(cartpole_rewards[-100:]):.2f}")

## Step 11: Test the Trained Agent

Now we test the trained agent to see how well it learned. The agent will use its learned policy without any more exploration (no random actions).

In [ ]:
# Test the trained agent for 10 episodes
print("Testing the trained agent...\n")

test_rewards = cartpole_agent.run(is_training=False, render=False, max_episodes=10)

print("\nTest Results:")
for i, reward in enumerate(test_rewards, 1):
    print(f"  Episode {i}: {reward:.2f} points")

print(f"\nAverage test reward: {np.mean(test_rewards):.2f}")
print(f"Best test reward: {max(test_rewards):.2f}")

## Step 12: Run Hyperparameter Tuning

This section performs systematic hyperparameter tuning using grid search. We test different combinations of learning rate, network size, and discount factor.

**Warning:** This will take a long time to run! Each combination trains for 100 episodes. You can reduce the number of combinations or episodes if needed.

In [ ]:
# Uncomment to run full hyperparameter tuning
# WARNING: This will take a long time!

'''
experiment_num = 1

for lr in learning_rates:
    for hidden in hidden_sizes:
        for gamma in discount_factors:
            print(f"\n{'='*60}")
            print(f"Experiment {experiment_num}")
            print(f"  Learning Rate: {lr}")
            print(f"  Hidden Size: {hidden}")
            print(f"  Discount Factor: {gamma}")
            print(f"{'='*60}")
            
            # Create modified hyperparameters
            test_params = cartpole_hyperparameters.copy()
            test_params['name'] = f'cartpole_exp{experiment_num}'
            test_params['learning_rate_a'] = lr
            test_params['fc1_nodes'] = hidden
            test_params['discount_factor_g'] = gamma
            
            # Train agent
            agent = Agent(test_params)
            rewards = agent.run(is_training=True, render=False, max_episodes=100)
            
            # Store results
            avg_reward = np.mean(rewards[-50:])  # Average of last 50 episodes
            experiment_results.append({
                'experiment': experiment_num,
                'learning_rate': lr,
                'hidden_size': hidden,
                'discount_factor': gamma,
                'avg_reward': avg_reward,
                'best_reward': max(rewards)
            })
            
            print(f"\nResults: Avg Reward = {avg_reward:.2f}, Best = {max(rewards):.2f}")
            experiment_num += 1

# Display results
import pandas as pd
results_df = pd.DataFrame(experiment_results)
print("\n" + "="*60)
print("HYPERPARAMETER TUNING RESULTS")
print("="*60)
print(results_df.to_string(index=False))

# Find best combination
best_idx = results_df['avg_reward'].idxmax()
best_config = results_df.iloc[best_idx]
print("\n" + "="*60)
print("BEST CONFIGURATION:")
print("="*60)
print(f"Learning Rate: {best_config['learning_rate']}")
print(f"Hidden Size: {best_config['hidden_size']}")
print(f"Discount Factor: {best_config['discount_factor']}")
print(f"Average Reward: {best_config['avg_reward']:.2f}")
'''

print("Hyperparameter tuning code is commented out.")
print("Uncomment the code above to run full grid search.")

## Step 13: Train on Flappy Bird

Flappy Bird is much harder than CartPole and requires many more episodes to learn. You can train it here if you have time.

**Note:** Flappy Bird may need 1000+ episodes to see good results. Consider training for a short time to see how it works, then training longer separately.

In [ ]:
# Uncomment to train Flappy Bird agent
# WARNING: This takes much longer than CartPole!

'''
print("Creating Flappy Bird agent...")
flappybird_agent = Agent(flappybird_hyperparameters)

print("\nStarting Flappy Bird training...")
print("This will take a while. Flappy Bird is a harder game!\n")

# Train for limited episodes (increase for better results)
flappybird_rewards = flappybird_agent.run(is_training=True, render=False, max_episodes=500)

print("\nFlappy Bird training completed!")
print(f"Final average reward: {np.mean(flappybird_rewards[-100:]):.2f}")
'''

print("Flappy Bird training code is commented out.")
print("Uncomment to train on Flappy Bird (will take much longer).")

## Summary and Conclusions

### What We Learned:

1. **Deep Q-Learning Basics**: We implemented a DQN agent that learns through trial and error

2. **Key Components**:
   - Experience Replay: Stores past experiences for better learning
   - Neural Network: The brain that makes decisions
   - Epsilon-Greedy: Balances exploration vs exploitation
   - Target Network: Provides stable learning targets

3. **Reward Functions**: The reward function defines what success means for the agent. Different reward structures lead to different learned behaviors.

4. **Hyperparameter Tuning**: Finding good hyperparameters is crucial for success. We showed how to systematically test different combinations using grid search.

### Important Hyperparameters:
- Learning rate: Controls learning speed
- Network architecture: Determines network capacity
- Discount factor: Balances immediate vs future rewards
- Epsilon decay: Controls exploration schedule

### Tips for Better Results:
1. Train for more episodes (CartPole: 500+, Flappy Bird: 2000+)
2. Try different network sizes based on problem complexity
3. Adjust reward functions if default rewards do not work well
4. Use Double DQN and Dueling DQN for harder problems
5. Monitor training graphs to catch problems early

### Next Steps:
- Experiment with custom reward functions
- Test more hyperparameter combinations
- Try other environments from Gymnasium
- Implement other RL algorithms (PPO, A3C, etc.)